In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
FPT = pd.read_csv('../data/sentiment/FPT_sentiment_qwen72b.csv')

In [13]:
FPT.count()

ticker                 3845
source                 3845
date                   3845
title                  3845
content                3845
url                    3845
year                   3845
related_tickers         424
target_score           3845
reason                 3845
full_sentiment_json    3845
dtype: int64

In [14]:
FPT[FPT['target_score']==0].count()

ticker                 2761
source                 2761
date                   2761
title                  2761
content                2761
url                    2761
year                   2761
related_tickers         373
target_score           2761
reason                 2761
full_sentiment_json    2761
dtype: int64

In [8]:
error=FPT[FPT['target_score']==0]

In [12]:
error.head(2)

,ticker,source,date,title,content,url,year,related_tickers,target_score,reason,full_sentiment_json
0,FPT,VnEconomy,2022-01-01,Thiếu hụt oxy trầm trọng tại các tỉnh phía Nam...,Để cung ứng kịp thời oxy cho các cơ sở y tế ch...,https://vneconomy.vn/thieu-hut-oxy-tram-trong-...,2022,NaN,0.0,"Bài báo không đề cập đến công ty FPT, không có...","{""FPT"": {""score"": 0, ""reason"": ""Bài báo không ..."
1,FPT,CafeF,2022-01-05,CTCP Tập đoàn Công nghệ CMC (Mã CK: CMG) đã th...,CTCP Tập đoàn Công nghệ CMC (Mã CK: CMG) đã th...,https://cafef.vn/cmc-group-cmg-thong-qua-phuon...,2022,NaN,0.0,Bài báo không đề cập đến thông tin liên quan đ...,"{""FPT"": {""score"": 0, ""reason"": ""Bài báo không ..."


In [16]:
FPT_final=pd.read_csv('../data/processed/FPT_final.csv')

In [18]:
FPT_final[FPT_final['related_tickers']=='CMG'].count()

ticker             120
source             120
date               120
title              120
content            120
url                120
year               120
related_tickers    120
dtype: int64

In [19]:
!pip install rapidfuzz

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 10.2 MB/s  0:00:00


In [25]:
from underthesea import ner
from rapidfuzz import process, fuzz

# 1. Map dữ liệu của bạn (Tên công ty -> Mã cổ phiếu)
# Lưu ý: Nên chuẩn bị danh sách key là các tên phổ biến
stock_map = {
    "Công ty Cổ phần FPT": "FPT",
    "Tập đoàn FPT": "FPT",
    "Công ty Cổ phần Đầu tư Thế Giới Di Động": "MWG",
    "Tập đoàn Vingroup": "VIC",
    "Ngân hàng TMCP Ngoại thương Việt Nam": "VCB",
    "Vietcombank": "VCB"
}

# 2. Hàm xử lý
def extract_tickers(text, mapping, threshold=80):
    # Bước 1: NER - Trích xuất thực thể
    # Underthesea trả về list các tuple (word, pos, tag, chunk)
    # Chúng ta chỉ quan tâm đến các tag là 'B-ORG' hoặc 'I-ORG' (Organization)
    
    entities = []
    tokens = ner(text)
    
    # Debug: In ra kết quả NER
    print("=== Kết quả NER ===")
    for token in tokens:
        print(token)
    print("=" * 30)
    
    current_entity = []
    for token in tokens:
        word, pos_tag, chunk_tag, ner_tag = token
        
        # Chấp nhận cả ORG, LOC, PER vì model có thể nhận diện sai
        if ner_tag != 'O':  # O = Outside (không phải entity)
            current_entity.append(word)
        else:
            if current_entity:
                entities.append(" ".join(current_entity))
                current_entity = []
    
    # Xử lý entity cuối cùng nếu còn sót
    if current_entity:
        entities.append(" ".join(current_entity))
    
    found_tickers = set()
    
    # Bước 2 & 3: Fuzzy Matching & Mapping
    # Lấy danh sách tên công ty từ map
    company_names = list(mapping.keys())
    
    for entity in entities:
        # Tìm tên công ty trong map giống nhất với entity trích xuất được
        match = process.extractOne(entity, company_names, scorer=fuzz.token_sort_ratio)
        
        if match:
            best_match_name, score, _ = match
            if score >= threshold:
                ticker = mapping[best_match_name]
                print(f"Tìm thấy: '{entity}' -> Map với: '{best_match_name}' ({score}%) -> Ticker: {ticker}")
                found_tickers.add(ticker)
        
        # Kiểm tra xem entity có phải là ticker trực tiếp không (VD: "FPT", "VCB")
        if entity.upper() in mapping.values():
            found_tickers.add(entity.upper())

    return list(found_tickers)

# --- CHẠY THỬ ---
text_news = """
Trong phiên giao dịch sáng nay, tập đoàn FPT đã công bố mức doanh thu kỷ lục. 
Trong khi đó, Thế giới di động và Vietcombank cũng ghi nhận tăng trưởng nhẹ.
"""

result = extract_tickers(text_news, stock_map)
print("-" * 20)
print("Danh sách mã cổ phiếu tìm thấy:", result)

=== Kết quả NER ===
('Trong', 'E', 'B-PP', 'O')
('phiên', 'N', 'B-NP', 'O')
('giao dịch', 'V', 'B-VP', 'O')
('sáng', 'N', 'B-NP', 'O')
('nay', 'P', 'B-NP', 'O')
(',', 'CH', 'O', 'O')
('tập đoàn', 'N', 'B-NP', 'B-LOC')
('FPT', 'Np', 'B-NP', 'I-LOC')
('đã', 'R', 'O', 'O')
('công bố', 'V', 'B-VP', 'O')
('mức', 'N', 'B-NP', 'O')
('doanh thu', 'N', 'B-NP', 'O')
('kỷ lục', 'N', 'B-NP', 'O')
('.', 'CH', 'O', 'O')
('Trong', 'E', 'B-PP', 'O')
('khi', 'N', 'B-NP', 'O')
('đó', 'P', 'B-NP', 'O')
(',', 'CH', 'O', 'O')
('Thế giới', 'N', 'B-NP', 'O')
('di động', 'V', 'B-VP', 'O')
('và', 'C', 'O', 'O')
('Vietcombank', 'Np', 'B-NP', 'B-PER')
('cũng', 'R', 'O', 'O')
('ghi nhận', 'V', 'B-VP', 'O')
('tăng trưởng', 'V', 'B-VP', 'O')
('nhẹ', 'A', 'B-AP', 'O')
('.', 'CH', 'O', 'O')
Tìm thấy: 'tập đoàn FPT' -> Map với: 'Tập đoàn FPT' (91.66666666666666%) -> Ticker: FPT
Tìm thấy: 'Vietcombank' -> Map với: 'Vietcombank' (100.0%) -> Ticker: VCB
--------------------
Danh sách mã cổ phiếu tìm thấy: ['FPT', 'VCB']
